In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn_extra.cluster import KMedoids
from scipy.spatial.distance import cdist

In [2]:
def splitMatrix(matrix):
    diagonalMatrix = np.diag(np.log(np.diag(matrix)))
    strictlyLowerTriangularMatrix = np.tril(matrix, k = -1)
    return diagonalMatrix, strictlyLowerTriangularMatrix

In [3]:
def distanceMatrices(matrix1, matrix2):
    chol1 = np.linalg.cholesky(matrix1)
    chol2 = np.linalg.cholesky(matrix2)
    chol1D, chol1L = splitMatrix(chol1)
    chol2D, chol2L = splitMatrix(chol2)
    distanceL = np.square(np.linalg.norm(chol1L - chol2L, 'fro'))
    distanceD = np.square(np.linalg.norm(chol1D - chol2D, 'fro'))
    distance = np.sqrt(distanceL + distanceD)
    return distance

In [4]:
def pairwiseDistances(matrixSet1, matrixSet2):
    matrixSet1 = matrixSet1.reshape(-1, 12, 12)
    matrixSet2 = matrixSet2.reshape(-1, 12, 12)
    shape1 = matrixSet1.shape[0]
    shape2 = matrixSet2.shape[0]
    PairwiseDistances = np.zeros((shape1, shape2))
    for i in range(shape1):
        for j in range(shape2):
            distance = distanceMatrices(matrixSet1[i], matrixSet2[j])
            PairwiseDistances[i, j] = distance
    return PairwiseDistances

In [5]:
covarianceMatrices = np.load("UCDMyoVerseSubject0Data.npy")
Labels = np.load("UCDMyoVerseSubject0Labels.npy")
print(covarianceMatrices.shape)
print(Labels.shape)

numberGestures = 10
trialsPerGesture = 36
numberChannels = 12
    
PairwiseDistances = cdist(covarianceMatrices.reshape(-1, 144), covarianceMatrices.reshape(-1, 144), metric = pairwiseDistances)
kmedoids = KMedoids(n_clusters = numberGestures, metric = "precomputed", method = 'pam', init = "k-medoids++", max_iter = 1000, random_state = 0)
kmedoids.fit(PairwiseDistances)

clusterLabels = kmedoids.labels_
medoidIndices = kmedoids.medoid_indices_
medoidLabels = Labels[medoidIndices]

assignedLabels = np.zeros((trialsPerGesture * numberGestures))
for i in range(trialsPerGesture * numberGestures):
    assignedLabels[i] = medoidLabels[clusterLabels[i]]

print("Mean acuracy is: ", np.mean(assignedLabels == Labels))

(360, 12, 12)
(360,)
Mean acuracy is:  0.9388888888888889
